In [ ]:

!pip -q install git+https://github.com/openai/CLIP.git opencv-python --upgrade

import os, math
import numpy as np
import torch, clip
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
from google.colab import files
import pandas as pd
import cv2

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)
USE_AMP = (device == "cuda")

MODEL_NAME = "ViT-L/14@336px"
TEMP       = 1.10
TARGET_FPS = 3.0
SAVE_TIMELINE_CSV = True

DECISION_MODE = "boxcar"

WINDOW_SEC = 1.0

SMOOTH_TAU_SEC = 5.0

HYSTERESIS    = 0.04
MIN_DWELL_SEC = 1.5

BACKDATE_SEC  = 0.8
CROSS_TOL     = 0.0

LAMBDA_SWITCH = 1.5

USE_TTA       = False
APPLY_ENHANCE = False

RELAX_BIAS             = 0.12
RELAX_UNCERT_MAXP      = 0.55
RELAX_NEAR_LOGIT_DIFF  = 0.80
RELAX_BIAS_DELTA_CAP   = 1.20

CHANGE_GAIN = 1.0
CHANGE_MIN  = 0.06

SIDE_GATE_ENABLE       = True
SIDE_NEAR_TIE_CONF     = 0.60
SIDE_NEAR_LOGIT_MARGIN = 0.45
SIDE_VIEW_THR          = 0.65
SIDE_MASK_QUALITY_MIN  = 0.50
SIDE_GAIN              = 0.70

ENABLE_TUBE_GUARD   = True
TUBE_CHECK_ONLY_NEAR = True
TUBE_NEAR_DELTA_P    = 0.20
TUBE_ALERT_THR       = 0.70
TUBE_LOGIT_MARGIN    = 0.50

TUBE_SPAN_MIN_DUR_SEC   = 0.50
TUBE_SPAN_MERGE_GAP_SEC = 0.25
TUBE_ALERT_FRAC         = 0.40

STRESS_RELAX_GEO_ENABLE = True
STRESS_RELAX_OPEN_THR   = 0.35
STRESS_RELAX_PUSH_P     = 0.60
STRESS_RELAX_PULL_P     = 0.40

model, preprocess = clip.load(MODEL_NAME, device=device)
model.eval()
LOGIT_SCALE = model.logit_scale.exp().detach().cpu().item()

VIEW_CTX = [
    "front view of a baby's hand, {}",
    "side view of a baby's hand, {}",
    "top-down view of a baby's hand, {}",
    "close-up of a baby's hand, {}",
    "a photo of a baby's hand, {}",
]
FIST_PHRASES = [
    "fist","tightly clenched fist","fingers curled","thumb tucked","knuckles prominent","closed hand"
]
SPLAY_PHRASES = [
    "fingers splayed","fingers widely spread","fully extended fingers, splayed",
    "maximally stretched hand","hyperextended fingers"
]
RELAXED_PHRASES = [
    "relaxed hand","natural resting hand","loose hand posture",
    "softly open hand","slightly open relaxed hand"
]

TUBE_CTX = [
    "front view of a baby's hand with {}",
    "side view of a baby's hand with {}",
    "top-down view of a baby's hand with {}",
    "close-up of a baby's hand with {}",
    "a photo of a baby's hand with {}",
]
TUBE_PHRASES = [
    "a medical tube","an IV line","a cannula","tubing","a catheter","a monitor cable",
    "adhesive tape securing a tube","a nasal cannula","a feeding tube","an oxygen tube"
]
PLAIN_HAND_PHRASES = [
    "bare baby's hand","plain baby's hand","unadorned baby's hand",
    "plain baby's hand on a clean background","natural baby's hand, skin visible",
    "close-up of a plain baby's hand"
]

def build_prompts(phrases, views):
    out=[]; seen=set()
    for v in views:
        for ph in phrases:
            s=v.format(ph); k=s.lower().strip()
            if k not in seen: out.append(s); seen.add(k)
    return out

PROMPTS_FIST    = build_prompts(FIST_PHRASES,   VIEW_CTX)
PROMPTS_SPLAY   = build_prompts(SPLAY_PHRASES,  VIEW_CTX)
PROMPTS_RELAXED = build_prompts(RELAXED_PHRASES,VIEW_CTX)
PROMPTS_TUBE    = build_prompts(TUBE_PHRASES,   TUBE_CTX)
PROMPTS_PLAIN   = build_prompts(PLAIN_HAND_PHRASES, TUBE_CTX)

@torch.no_grad()
def encode_texts(texts):
    toks = clip.tokenize(texts, truncate=True).to(device)
    feats = model.encode_text(toks)
    feats = feats / feats.norm(dim=-1, keepdim=True)
    return feats

text_fist    = encode_texts(PROMPTS_FIST)
text_splay   = encode_texts(PROMPTS_SPLAY)
text_relaxed = encode_texts(PROMPTS_RELAXED)
text_tube    = encode_texts(PROMPTS_TUBE)
text_plain   = encode_texts(PROMPTS_PLAIN)

def enhance_optional(pil_img):
    if not APPLY_ENHANCE: return pil_img
    arr = np.array(pil_img.convert("RGB"))
    lab = cv2.cvtColor(arr, cv2.COLOR_RGB2LAB)
    L,a,b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    L2 = clahe.apply(L)
    lab2 = cv2.merge([L2,a,b])
    rgb2 = cv2.cvtColor(lab2, cv2.COLOR_LAB2RGB)
    rgb2 = cv2.fastNlMeansDenoisingColored(rgb2, None, 3,3,7,21)
    return Image.fromarray(rgb2)

def tta_images(pil_img):
    base = ImageOps.exif_transpose(pil_img)
    if not USE_TTA: return [base]
    return [base,
            base.transpose(Image.ROTATE_90),
            base.transpose(Image.ROTATE_180),
            base.transpose(Image.ROTATE_270)]

@torch.no_grad()
def embed_image(pil_img):
    x = preprocess(pil_img).unsqueeze(0).to(device)
    if USE_AMP:
        with torch.cuda.amp.autocast():
            f = model.encode_image(x)
    else:
        f = model.encode_image(x)
    f = f / f.norm(dim=-1, keepdim=True)
    return f.squeeze(0)


def _largest_contour(mask):
    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts: return None
    c = max(cnts, key=cv2.contourArea)
    if cv2.contourArea(c) < 300: return None
    return c

def _hand_mask_quick(pil_img):
    im  = np.array(pil_img.convert("RGB"))
    gray= cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
    gray= cv2.GaussianBlur(gray,(5,5),0)
    gray= cv2.equalizeHist(gray)
    _, th = cv2.threshold(gray, 0,255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    if th.mean() < 127: th = cv2.bitwise_not(th)
    return th

def side_view_score_and_quality(pil_img):
    th = _hand_mask_quick(pil_img)
    c  = _largest_contour(th)
    if c is None: return 0.0, 0.0

    rect = cv2.minAreaRect(c); (w,h) = rect[1]
    if w<1 or h<1: return 0.0, 0.0
    ar = max(w,h)/max(1.0,min(w,h))
    ar_score = np.tanh((ar-1.5)/0.6)
    ar_score = float(np.clip(ar_score,0.0,1.0))

    hull = cv2.convexHull(c)
    area = cv2.contourArea(c) + 1e-6
    harea= cv2.contourArea(hull) + 1e-6
    solidity = float(np.clip(area/harea, 0.0, 1.0))
    return ar_score, solidity

def geo_tiebreak_side(pil_img):
    th = _hand_mask_quick(pil_img)
    c  = _largest_contour(th)
    if c is None: return 0.0

    per   = cv2.arcLength(c, True) + 1e-6
    area  = cv2.contourArea(c)
    compact = 4*np.pi*area/(per*per)
    compact = float(np.clip((compact-0.45)/0.25, 0.0, 1.0))

    hull = cv2.convexHull(c, returnPoints=False)
    defects = cv2.convexityDefects(c, hull) if hull is not None and len(hull)>=3 else None
    gaps = 0 if defects is None else int((defects[:,:,3][:,0]/256.0 > 8).sum())
    gaps = float(np.clip(gaps/6.0, 0.0, 1.0))

    g = float(np.clip(compact - gaps, -1.0, 1.0))
    return g
def side_openness_score(pil_img):

    th = _hand_mask_quick(pil_img)
    c  = _largest_contour(th)
    if c is None:
        return None

    hull = cv2.convexHull(c, returnPoints=False)
    if hull is None or len(hull) < 3:
        return None

    defects = cv2.convexityDefects(c, hull)
    if defects is None:
        defects_count = 0
    else:
        depths = defects[:,:,3][:,0] / 256.0
        defects_count = int((depths > 10).sum())

    area  = cv2.contourArea(c)
    perim = cv2.arcLength(c, True) + 1e-6
    compact = 4 * np.pi * area / (perim * perim)

    open_score = 0.5 * min(1.0, defects_count / 5.0) + 0.5 * (1.0 - np.clip(compact, 0, 1))
    return float(open_score)


@torch.no_grad()
def tube_presence_prob(pil_img, temp=TEMP):
    pil_img = enhance_optional(pil_img)
    feats = [embed_image(im) for im in tta_images(pil_img)]
    f = torch.stack(feats, dim=0).mean(dim=0)

    Lt = (f @ text_tube.T)   * LOGIT_SCALE / temp
    Lp = (f @ text_plain.T)  * LOGIT_SCALE / temp

    St = torch.logsumexp(Lt, dim=-1)
    Sp = torch.logsumexp(Lp, dim=-1)

    pp = torch.stack([St, Sp], dim=0).softmax(dim=0)
    p_tube  = float(pp[0].detach().cpu().numpy())
    p_plain = float(pp[1].detach().cpu().numpy())
    margin  = float(abs(St - Sp))
    return p_tube, p_plain, float(St), float(Sp), margin

@torch.no_grad()
def triple_probs_raw(pil_img, temp=TEMP):
    pil_img = enhance_optional(pil_img)
    feats = [embed_image(im) for im in tta_images(pil_img)]
    f = torch.stack(feats, dim=0).mean(dim=0)

    Lf = (f @ text_fist.T)    * LOGIT_SCALE / temp
    Ls = (f @ text_splay.T)   * LOGIT_SCALE / temp
    Lr = (f @ text_relaxed.T) * LOGIT_SCALE / temp

    Sf = torch.logsumexp(Lf, dim=-1)
    Ss = torch.logsumexp(Ls, dim=-1)
    Sr = torch.logsumexp(Lr, dim=-1)

    pair = torch.stack([Sf, Ss], dim=0)
    maxp_stress = pair.softmax(dim=0).max().item()
    near_logits  = abs(float(Sf - Ss)) < RELAX_NEAR_LOGIT_DIFF
    not_crushed  = (float(max(Sf,Ss)) - float(Sr)) < RELAX_BIAS_DELTA_CAP
    if (maxp_stress < RELAX_UNCERT_MAXP) and near_logits and not_crushed:
        Sr = Sr + RELAX_BIAS

    if SIDE_GATE_ENABLE:
        conf_pair = pair.softmax(dim=0).max().item()
        near_tie  = (conf_pair < SIDE_NEAR_TIE_CONF) or (abs(float(Sf - Ss)) < SIDE_NEAR_LOGIT_MARGIN)
        if near_tie:
            side_score, quality = side_view_score_and_quality(pil_img)
            if side_score >= SIDE_VIEW_THR and quality >= SIDE_MASK_QUALITY_MIN:
                g = geo_tiebreak_side(pil_img)
                if abs(g) > 1e-6:
                    Sf = Sf + SIDE_GAIN * g
                    Ss = Ss - SIDE_GAIN * g

    vec = torch.stack([Sf, Ss, Sr], dim=0)
    p = vec.softmax(dim=0)
    p_fist, p_splay, p_relaxed = map(float, p.detach().cpu().numpy())
    idx = int(np.argmax([p_fist, p_splay, p_relaxed]))
    label = ["FIST","SPLAY","RELAXED"][idx]
    return p_fist, p_splay, p_relaxed, label, float(Sf), float(Ss), float(Sr)


def video_iter(path, target_fps=TARGET_FPS):
    cap = cv2.VideoCapture(path)
    if not cap.isOpened(): raise RuntimeError(f"Nem tudtam megnyitni: {path}")
    src_fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
    stride  = max(1, int(round(src_fps / float(target_fps))))
    idx = 0
    while True:
        ok, frame_bgr = cap.read()
        if not ok: break
        if idx % stride == 0:
            t_sec = idx / src_fps
            frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
            pil_img = Image.fromarray(frame_rgb).convert("RGB")
            yield idx, t_sec, pil_img
        idx += 1
    cap.release()

def fmt_ts(seconds: float) -> str:
    seconds = max(0, float(seconds)); m = int(seconds//60); s = int(round(seconds%60))
    if s==60: s=0; m+=1
    return f"{m:02d}:{s:02d}"


def _fps_from_t(t):
    if len(t) < 2: return TARGET_FPS
    dt = np.diff(t)
    med = np.median(dt)
    return 1.0 / max(1e-6, med)

def _sliding_mean(x, win):
    if win <= 1: return x.copy()
    c = np.cumsum(np.insert(x, 0, 0.0))
    y = (c[win:] - c[:-win]) / float(win)
    left  = np.full(win//2, y[0])
    right = np.full(win - 1 - win//2, y[-1])
    return np.concatenate([left, y, right])

def _sliding_median(x, win):
    if win <= 1: return x.copy()
    y = np.empty_like(x)
    half = win//2
    for i in range(len(x)):
        a = max(0, i - half)
        b = min(len(x), i + half + 1)
        y[i] = np.median(x[a:b])
    return y

def _ema_time(t, x, tau):
    x = np.asarray(x, dtype=float)
    if len(t) < 2: return x.copy()
    y = np.empty_like(x)
    y[0] = x[0]
    for i in range(1, len(t)):
        a = 1.0 - math.exp(-(t[i]-t[i-1]) / max(1e-6, tau))
        y[i] = (1-a)*y[i-1] + a*x[i]
    return y

def _segments_from_state(t, state_arr, backdate_sec=0.0, cross_tol=0.0,
                         stress_raw=None, relax_raw=None):
    segs=[]; cur = state_arr[0]; seg_start = t[0]
    for i in range(1, len(t)):
        if state_arr[i] != cur:
            t_change = t[i]
            if backdate_sec > 0.0 and stress_raw is not None and relax_raw is not None:
                t_lim = t[i] - backdate_sec
                k = i
                if state_arr[i]:
                    while k > 0 and t[k] >= t_lim and (stress_raw[k] >= relax_raw[k] + cross_tol):
                        k -= 1
                else:
                    while k > 0 and t[k] >= t_lim and (relax_raw[k] >= stress_raw[k] + cross_tol):
                        k -= 1
                t_change = t[max(0,k)]
            segs.append((seg_start, t_change, "STRESSZ" if cur else "RELAXÁLT"))
            cur = state_arr[i]; seg_start = t_change
    segs.append((seg_start, t[-1], "STRESSZ" if cur else "RELAXÁLT"))
    return segs

def _apply_min_dwell(segs, min_dwell):
    if min_dwell <= 0 or len(segs) <= 1: return segs
    out=[segs[0]]
    for (t0,t1,lab) in segs[1:]:
        if (t1 - t0) < min_dwell and out and out[-1][2] != lab:
            prev_t0, prev_t1, prev_lab = out[-1]
            out[-1] = (prev_t0, t1, prev_lab)
        else:
            out.append((t0,t1,lab))
    return out

def _viterbi_two_state(t, p_stress, p_relax, lambda_switch):
    eps = 1e-6
    eS = np.log(np.clip(p_stress, eps, 1.0))
    eR = np.log(np.clip(p_relax,  eps, 1.0))

    n = len(t)
    dpS = np.empty(n); dpR = np.empty(n)
    bpS = np.zeros(n, dtype=np.uint8); bpR = np.zeros(n, dtype=np.uint8)

    dpS[0] = eS[0]; dpR[0] = eR[0]
    for i in range(1, n):
        stayS   = dpS[i-1]
        switchS = dpR[i-1] - lambda_switch
        if stayS >= switchS:
            dpS[i] = stayS; bpS[i] = 0
        else:
            dpS[i] = switchS; bpS[i] = 1

        stayR   = dpR[i-1]
        switchR = dpS[i-1] - lambda_switch
        if stayR >= switchR:
            dpR[i] = stayR; bpR[i] = 0
        else:
            dpR[i] = switchR; bpR[i] = 1

        dpS[i] += eS[i]
        dpR[i] += eR[i]

    st = np.zeros(n, dtype=bool)
    cur = 1 if dpR[-1] > dpS[-1] else 0
    for i in range(n-1, -1, -1):
        st[i] = (cur == 0)
        if cur == 0:
            cur = 1 if bpS[i] == 1 else 0
        else:
            cur = 0 if bpR[i] == 1 else 1
    return st

def segment_decision(t, pf, ps, pr,
                     mode="raw_hard",
                     window_sec=1.0,
                     tau_sec=5.0,
                     hysteresis=0.04,
                     min_dwell_sec=1.5,
                     backdate_sec=0.8,
                     cross_tol=0.0,
                     lambda_switch=1.5,
                     stress_override=None,
                     relax_override=None):
    if stress_override is None:
        stress_raw = np.maximum(pf, ps).astype(float)
    else:
        stress_raw = np.asarray(stress_override, dtype=float)
    if relax_override is None:
        relax_raw  = pr.astype(float)
    else:
        relax_raw  = np.asarray(relax_override, dtype=float)

    stress_disp = stress_raw.copy()
    relax_disp  = relax_raw.copy()

    if mode == "raw_hard":
        state = stress_raw >= relax_raw
        segs = _segments_from_state(t, state, backdate_sec=0.0, cross_tol=0.0)
        return segs, stress_disp, relax_disp, state

    if mode == "hysteresis_only":
        d = stress_raw - relax_raw
        state = np.zeros_like(d, dtype=bool)
        state[0] = d[0] >= 0
        last_switch_t = t[0]
        for i in range(1, len(t)):
            cur = state[i-1]
            if cur:
                if (d[i] < -hysteresis) and (t[i]-last_switch_t >= min_dwell_sec):
                    state[i] = False; last_switch_t = t[i]
                else:
                    state[i] = True
            else:
                if (d[i] > +hysteresis) and (t[i]-last_switch_t >= min_dwell_sec):
                    state[i] = True; last_switch_t = t[i]
                else:
                    state[i] = False
        segs = _segments_from_state(t, state, backdate_sec=backdate_sec,
                                    cross_tol=cross_tol,
                                    stress_raw=stress_raw, relax_raw=relax_raw)
        segs = _apply_min_dwell(segs, min_dwell_sec)
        return segs, stress_disp, relax_disp, state

    if mode == "ema":
        stress_disp = _ema_time(t, stress_raw, tau_sec)
        relax_disp  = _ema_time(t, relax_raw,  tau_sec)
        state = stress_disp >= relax_disp
        segs = _segments_from_state(t, state, backdate_sec=backdate_sec,
                                    cross_tol=cross_tol,
                                    stress_raw=stress_raw, relax_raw=relax_raw)
        segs = _apply_min_dwell(segs, min_dwell_sec)
        return segs, stress_disp, relax_disp, state

    fps = _fps_from_t(t)
    win = max(1, int(round(window_sec * fps)))
    if win % 2 == 0: win += 1

    if mode == "median":
        stress_disp = _sliding_median(stress_raw, win)
        relax_disp  = _sliding_median(relax_raw,  win)
        state = stress_disp >= relax_disp
        segs = _segments_from_state(t, state, backdate_sec=backdate_sec,
                                    cross_tol=cross_tol,
                                    stress_raw=stress_raw, relax_raw=relax_raw)
        segs = _apply_min_dwell(segs, min_dwell_sec)
        return segs, stress_disp, relax_disp, state

    if mode == "boxcar":
        stress_disp = _sliding_mean(stress_raw, win)
        relax_disp  = _sliding_mean(relax_raw,  win)
        state = stress_disp >= relax_disp
        segs = _segments_from_state(t, state, backdate_sec=backdate_sec,
                                    cross_tol=cross_tol,
                                    stress_raw=stress_raw, relax_raw=relax_raw)
        segs = _apply_min_dwell(segs, min_dwell_sec)
        return segs, stress_disp, relax_disp, state

    if mode == "vote":
        raw_state = (stress_raw >= relax_raw).astype(np.int8)
        voted = np.zeros_like(raw_state)
        half = win//2
        for i in range(len(raw_state)):
            a = max(0, i-half); b=min(len(raw_state), i+half+1)
            s = raw_state[a:b].sum()
            voted[i] = 1 if s*2 >= (b-a) else 0
        state = voted.astype(bool)
        segs = _segments_from_state(t, state, backdate_sec=backdate_sec,
                                    cross_tol=cross_tol,
                                    stress_raw=stress_raw, relax_raw=relax_raw)
        segs = _apply_min_dwell(segs, min_dwell_sec)
        return segs, stress_disp, relax_disp, state

    if mode == "hmm":
        state = _viterbi_two_state(t, stress_raw, relax_raw, lambda_switch)
        segs = _segments_from_state(t, state, backdate_sec=backdate_sec,
                                    cross_tol=cross_tol,
                                    stress_raw=stress_raw, relax_raw=relax_raw)
        segs = _apply_min_dwell(segs, min_dwell_sec)
        return segs, stress_disp, relax_disp, state

    raise ValueError(f"Ismeretlen DECISION_MODE: {mode}")


def _spans_from_bool(t, mask_bool):
    spans=[]
    if len(t)==0: return spans
    cur = bool(mask_bool[0])
    start = t[0]
    for i in range(1,len(t)):
        if bool(mask_bool[i]) != cur:
            if cur:
                spans.append((start, t[i]))
            start = t[i]
            cur = bool(mask_bool[i])
    if cur:
        spans.append((start, t[-1]))
    return spans

def _filter_spans(spans, min_dur):
    if min_dur <= 0: return spans[:]
    return [(a,b) for (a,b) in spans if (b-a) >= min_dur]

def _merge_close_spans(spans, min_gap):
    if not spans: return []
    spans = sorted(spans, key=lambda x: x[0])
    out = [spans[0]]
    for (a,b) in spans[1:]:
        A,B = out[-1]
        if a - B <= max(0.0, min_gap):
            out[-1] = (A, max(B,b))
        else:
            out.append((a,b))
    return out

print("Válassz ki egy vagy több videófájlt (pl. .mp4, .mov, .avi).")
uploaded = files.upload()
os.makedirs("video_inputs", exist_ok=True)

for vname, vbytes in uploaded.items():
    vpath = os.path.join("video_inputs", vname)
    with open(vpath, "wb") as f: f.write(vbytes)

    rows=[]
    prev_fist = prev_splay = None
    any_tube = False
    print(f"\n--- Feldolgozás: {vname} ---  MODEL={MODEL_NAME}, TARGET_FPS={TARGET_FPS}")

    for i,(fi,t_sec,pil_img) in enumerate(video_iter(vpath, TARGET_FPS)):
        p_fist, p_splay, p_relaxed, label, Sf, Ss, Sr = triple_probs_raw(pil_img)

        stress_prob = max(p_fist, p_splay)
        relax_prob  = p_relaxed
        near_tie = abs(stress_prob - relax_prob) < TUBE_NEAR_DELTA_P

        stress_prob_geo = stress_prob
        relax_prob_geo  = relax_prob
        sr_geom_label   = ""

        p_tube = p_plain = 0.0
        St = Sp = 0.0
        tube_flag = False
        tube_checked = False
        if ENABLE_TUBE_GUARD and ((not TUBE_CHECK_ONLY_NEAR) or near_tie):
            tube_checked = True
            p_tube, p_plain, St, Sp, margin = tube_presence_prob(pil_img)
            tube_flag = (p_tube >= TUBE_ALERT_THR) and ((St - Sp) >= TUBE_LOGIT_MARGIN)
            any_tube = any_tube or tube_flag
        else:
            margin = 0.0

        if (STRESS_RELAX_GEO_ENABLE
            and near_tie
            and (not tube_flag)):
            side_score, quality = side_view_score_and_quality(pil_img)
            if side_score >= SIDE_VIEW_THR and quality >= SIDE_MASK_QUALITY_MIN:
                open_sc = side_openness_score(pil_img)
                if (open_sc is not None) and (open_sc >= STRESS_RELAX_OPEN_THR):
                    relax_prob_geo  = max(relax_prob_geo,  STRESS_RELAX_PUSH_P)
                    stress_prob_geo = min(stress_prob_geo, STRESS_RELAX_PULL_P)
                    sr_geom_label   = "RELAXÁLT (geom TB)"

        if prev_fist is None:
            change = 0.0
        else:
            dfv = p_fist  - prev_fist
            dsv = p_splay - prev_splay
            cross = max(0.0, dfv * (-dsv))
            mag   = min(1.0, math.sqrt(dfv*dfv + dsv*dsv))
            change = CHANGE_GAIN * (0.7*cross + 0.3*mag)
            if change < CHANGE_MIN:
                change = 0.0
        prev_fist, prev_splay = p_fist, p_splay

        if tube_flag:
            label_effective = "NEM ÉRTÉKELHETŐ (CSŐ)"
        else:
            label_effective = label
            if sr_geom_label:
                label_effective = f"{label_effective} | {sr_geom_label}"

        rows.append({
            "frame_index": fi, "t_sec": t_sec, "t_str": fmt_ts(t_sec),
            "p_fist": p_fist, "p_splay": p_splay, "p_relaxed": p_relaxed,
            "label_raw": label, "label_effective": label_effective,
            "Sf": Sf, "Ss": Ss, "Sr": Sr,
            "change_score": change,
            "p_tube": p_tube, "p_plain": p_plain, "tube_margin": margin,
            "tube_flag": bool(tube_flag), "tube_checked": bool(tube_checked),
            "near_tie": bool(near_tie),
            "stress_prob_base": stress_prob,
            "relax_prob_base":  relax_prob,
            "stress_prob_geo":  stress_prob_geo,
            "relax_prob_geo":   relax_prob_geo,
            "sr_geom_label":    sr_geom_label,
        })
        if (i+1) % 50 == 0: print(f"  minták feldolgozva: {i+1}")

    if not rows:
        print("Nincs minta ebben a videóban.")
        continue

    df = pd.DataFrame(rows)

    t  = df["t_sec"].to_numpy()
    pf = df["p_fist"].to_numpy()
    ps = df["p_splay"].to_numpy()
    pr = df["p_relaxed"].to_numpy()

    stress_geo_arr = df["stress_prob_geo"].to_numpy()
    relax_geo_arr  = df["relax_prob_geo"].to_numpy()

    segs, stress_disp, relax_disp, state = segment_decision(
        t, pf, ps, pr,
        mode=DECISION_MODE,
        window_sec=WINDOW_SEC,
        tau_sec=SMOOTH_TAU_SEC,
        hysteresis=HYSTERESIS,
        min_dwell_sec=MIN_DWELL_SEC,
        backdate_sec=BACKDATE_SEC if DECISION_MODE != "raw_hard" else 0.0,
        cross_tol=CROSS_TOL,
        lambda_switch=LAMBDA_SWITCH,
        stress_override=stress_geo_arr,
        relax_override=relax_geo_arr,
    )

    stress_raw = stress_geo_arr
    relax_raw  = relax_geo_arr

    tube_mask = df["tube_flag"].to_numpy().astype(bool)

    tube_spans_raw = _spans_from_bool(t, tube_mask)
    tube_spans_merged = _merge_close_spans(tube_spans_raw, TUBE_SPAN_MERGE_GAP_SEC)
    tube_spans = _filter_spans(tube_spans_merged, TUBE_SPAN_MIN_DUR_SEC)


    fig, ax = plt.subplots(2, 1, figsize=(11,6), sharex=True)
    ax[0].plot(t, pf, label="fist (raw)")
    ax[0].plot(t, ps, label="splay (raw)")
    ax[0].plot(t, pr, label="relaxed (raw)")
    for (t0, t1, lab) in segs:
        ax[0].axvspan(t0, t1, color=(0.95,0.60,0.60,0.20) if lab=="STRESSZ" else (0.60,0.85,0.95,0.20))
    for (a,b) in tube_spans:
        ax[0].axvspan(a, b, color=(0.3,0.3,0.3,0.12))
    ax[0].set_ylim(0,1); ax[0].legend(loc="upper right")
    ax[0].set_title(f"{vname} – RAW probs (bg: {DECISION_MODE} állapot, szürke: CSŐ)")

    ax[1].plot(t, stress_raw,  alpha=0.25, linewidth=1.0,
              label="stress_raw (geo)",  color="red")
    ax[1].plot(t, relax_raw,   alpha=0.25, linewidth=1.0,
              label="relax_raw (geo)",   color="green")

    ax[1].plot(t, stress_disp, linewidth=2.0,
              label=f"stress_{DECISION_MODE}", color="red")
    ax[1].plot(t, relax_disp,  linewidth=2.0,
              label=f"relax_{DECISION_MODE}",  color="green")

    for (t0, t1, lab) in segs:
        ax[1].axvspan(t0, t1, color=(0.95,0.60,0.60,0.12) if lab=="STRESSZ" else (0.60,0.85,0.95,0.12))
    for (a,b) in tube_spans:
        ax[1].axvspan(a, b, color=(0.3,0.3,0.3,0.08))
    ax[1].set_ylim(0,1); ax[1].legend(loc="upper right")
    ax[1].set_xlabel("t [s]"); ax[1].set_ylabel("p")
    plt.tight_layout(); plt.show()

    df["stress_raw"]   = stress_raw
    df["relax_raw"]    = relax_raw
    df["stress_disp"]  = stress_disp
    df["relax_disp"]   = relax_disp
    df["state"]        = np.where(state, "STRESSZ", "RELAXÁLT")

    perframe_csv = os.path.splitext(vpath)[0] + "_perframe.csv"
    df.to_csv(perframe_csv, index=False)
    print(f"Mentve: {os.path.basename(perframe_csv)}")

    seg_rows=[]
    print("\nIdővonal:")
    for (t0,t1,lab) in segs:
        print(f"  {fmt_ts(t0)} – {fmt_ts(t1)}  {lab}  (dur {t1-t0:.2f}s)")
        seg_rows.append({
            "start_sec": t0, "end_sec": t1, "start_ts": fmt_ts(t0), "end_ts": fmt_ts(t1),
            "label": lab, "duration_sec": max(0.0, t1-t0)
        })
    seg_df_out = pd.DataFrame(seg_rows)
    if SAVE_TIMELINE_CSV:
        seg_csv = os.path.splitext(vpath)[0] + "_timeline.csv"
        seg_df_out.to_csv(seg_csv, index=False)
        print(f"Mentve: {os.path.basename(seg_csv)}")

    if tube_spans:
        tube_rows=[]
        for (a,b) in tube_spans:
            tube_rows.append({
                "start_sec": a, "end_sec": b, "start_ts": fmt_ts(a), "end_ts": fmt_ts(b),
                "label": "CSŐ / NEM ÉRTÉKELHETŐ", "duration_sec": max(0.0, b-a)
            })
        tube_df = pd.DataFrame(tube_rows)
        tube_csv = os.path.splitext(vpath)[0] + "_tube_timeline.csv"
        tube_df.to_csv(tube_csv, index=False)
        print(f"Mentve: {os.path.basename(tube_csv)}")

    tot_stress = sum(max(0.0, t1-t0) for (t0,t1,lab) in segs if lab=="STRESSZ")
    tot_relax  = sum(max(0.0, t1-t0) for (t0,t1,lab) in segs if lab=="RELAXÁLT")
    print("\nÖsszegzés:")
    print(f"  STRESSZ összesen: {tot_stress:.2f} s")
    print(f"  RELAXÁLT összesen: {tot_relax:.2f} s")
    if segs:
        first_lab = segs[0][2]; last_lab = segs[-1][2]
        print(f"  Kezdő állapot: {first_lab}, záró állapot: {last_lab}")


    video_dur = max(1e-6, t[-1] - t[0])
    tube_total = sum(max(0.0, b-a) for (a,b) in tube_spans)
    tube_frac  = tube_total / video_dur

    if tube_total > 0:
        print(f"  FIGYELEM: cső/kábel észlelve ~ {tube_total:.2f} s ({tube_frac*100:.1f}%). Ezekben a szakaszokban nem adok releváns döntést.")

    if tube_frac >= TUBE_ALERT_FRAC:
        print(f"  FIGYELMEZTETÉS (VIDEÓ): csőgyanús szakaszok aránya ≥ {int(TUBE_ALERT_FRAC*100)}%.")


    dbg_err = float(np.mean(np.abs((1.0 - stress_raw) - relax_raw)))
    print(f"DEBUG: mean(|(1-stress_raw) - relax_raw|) = {dbg_err:.4f}  (nem várjuk, hogy ~0 legyen)")
